In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from mlxtend.frequent_patterns import apriori, association_rules


ModuleNotFoundError: No module named 'mlxtend'

In [2]:
# Load dataset
df = pd.read_excel("OnlineRetail (1).xlsx")

# Drop missing customer IDs
df = df.dropna(subset=['CustomerID'])

# Remove cancellations
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Remove invalid rows
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Convert to correct types
df['CustomerID'] = df['CustomerID'].astype(int)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'OnlineRetail (1).xlsx'

In [3]:
# Top 10 most sold products
top_products = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,5))
sns.barplot(x=top_products.values, y=top_products.index, palette="viridis")
plt.title("Top 10 Popular Products by Quantity")
plt.xlabel("Units Sold")
plt.ylabel("Product")
plt.show()

top_products


NameError: name 'df' is not defined

In [ ]:
# Build Customer–Item matrix
user_item = df.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', aggfunc='sum', fill_value=0)

# Compute cosine similarity between items
item_sim = cosine_similarity(user_item.T)
item_sim_df = pd.DataFrame(item_sim, index=user_item.columns, columns=user_item.columns)

# Pick a random customer
customer_id = df['CustomerID'].iloc[0]
print("Customer ID:", customer_id)

# Items bought by this customer
customer_items = user_item.loc[customer_id]
owned = customer_items[customer_items > 0].index.tolist()
print("Products bought:", owned)

# Recommend similar items
scores = item_sim_df[owned].mean(axis=1).sort_values(ascending=False)
scores = scores.drop(owned)[:10]
scores


In [ ]:
# Build baskets (InvoiceNo × Product)
basket = (df.groupby(['InvoiceNo','Description'])['Quantity']
            .sum().unstack().fillna(0))
basket = (basket > 0).astype(int)

# Apply Apriori
frequent = apriori(basket, min_support=0.002, use_colnames=True)
rules = association_rules(frequent, metric="lift", min_threshold=1)

rules.sort_values('lift', ascending=False).head(10)


In [4]:
product = top_products.index[0]  # Most popular product
print("Anchor Product:", product)

related = rules[rules['antecedents'].apply(lambda x: product in x)]
related[['antecedents','consequents','support','confidence','lift']].head(10)


NameError: name 'top_products' is not defined